## Performance improvements

* Invoked sparsity throughout analysis: significant speedup
* Re-wrote QCQP matrix generation methods, 90% reduction in memory allocation
* Wrapped core computation in parallelizable for loop
* Plotted computation time vs memory allocation for various numbers of worker processes *(plot below)*

![Parallel performance](../images/2015-07-29-parallelization-performance.svg)

* Re-wrote `find_x_star` to handle cases where $A$ doesn't have full row rank
* Added zero-resistance check: if a line has zero resistance, no amount of power flow can influence its temperature, so skip it.
* Created type containers for input data and output data; makes it easier to load data for any network or plot any analysis output regardless of conductor parameters.
* ** 6-time-step RTS-96 analysis time: **

| Time (s) | Note                        |
|:--------:|:----------------------------|
| 420      | Original                    |
| 20       | No more `backslash`         |
| 13       | Invoked sparsity            |
| 8        | Fine-tuning                 |
| 4        | Parallelize to 2 processors |

## Testing on multiple networks

* Wrote code to supplement data for any Matpower network with RTS-96 thermal data
* Performed instanton analysis on nine Matpower networks (with RTS-96-esque thermal data) to characterize performance *(plot below)* 

![Performance by network](../images/2015-08-11-performance-by-network.svg)

## Covariance matrices

* Pass [randomly-generated][1] unit-norm covariance matrix to instanton analysis code.
* Observed no discernible performance implications.
* Observed downward shift in instanton candidate scores for 6-time-step RTS-96 analysis *(plot below)*

![Score by covariance](../images/2015-08-11-score-by-covariance.svg)

[1]: http://dx.doi.org/10.1016/j.jmva.2009.04.008

## Roadmap

* Continue writing about findings, refining existing draft
* Better visualization of covariance effects
* Finer control of covariance matrix generation
* Study temporal covariance
* Attempt to use `eigs` rather than `eig` to speed up computation on larger networks

In [1]:
?eigs

search: 

```rst
::
           eigs(A, [B,]; nev=6, which="LM", tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)

Computes eigenvalues ``d`` of ``A`` using Lanczos or Arnoldi iterations for
real symmetric or general nonsymmetric matrices respectively. If ``B`` is
provided, the generalized eigenproblem is solved.

The following keyword arguments are supported:
 * ``nev``: Number of eigenvalues
 * ``ncv``: Number of Krylov vectors used in the computation; should satisfy
    ``nev+1 <= ncv <= n`` for real symmetric problems and ``nev+2 <= ncv <= n``
    for other problems, where ``n`` is the size of the input matrix ``A``.
    The default is ``ncv = max(20,2*nev+1)``.
    Note that these restrictions limit the input matrix ``A`` to be of
    dimension at least 2.
 * ``which``: type of eigenvalues to compute. See the note below.

   ========= ======================================================================================================================
   ``which`` type of eigenvalues
   --------- ----------------------------------------------------------------------------------------------------------------------
   ``:LM``   eigenvalues of largest magnitude (default)
   ``:SM``   eigenvalues of smallest magnitude
   ``:LR``   eigenvalues of largest real part
   ``:SR``   eigenvalues of smallest real part
   ``:LI``   eigenvalues of largest imaginary part (nonsymmetric or complex ``A`` only)
   ``:SI``   eigenvalues of smallest imaginary part (nonsymmetric or complex ``A`` only)
   ``:BE``   compute half of the eigenvalues from each end of the spectrum, biased in favor of the high end. (real symmetric ``A`` only)
   ========= ======================================================================================================================

 * ``tol``: tolerance (:math:`tol \le 0.0` defaults to ``DLAMCH('EPS')``)
 * ``maxiter``: Maximum number of iterations (default = 300)
 * ``sigma``: Specifies the level shift used in inverse iteration. If ``nothing`` (default), defaults to ordinary (forward) iterations. Otherwise, find eigenvalues close to ``sigma`` using shift and invert iterations.
 * ``ritzvec``: Returns the Ritz vectors ``v`` (eigenvectors) if ``true``
 * ``v0``: starting vector from which to start the iterations

``eigs`` returns the ``nev`` requested eigenvalues in ``d``, the corresponding Ritz vectors ``v`` (only if ``ritzvec=true``), the number of converged eigenvalues ``nconv``, the number of iterations ``niter`` and the number of matrix vector multiplications ``nmult``, as well as the final residual vector ``resid``.

.. note:: The ``sigma`` and ``which`` keywords interact: the description of eigenvalues searched for by ``which`` do _not_ necessarily refer to the eigenvalues of ``A``, but rather the linear operator constructed by the specification of the iteration mode implied by ``sigma``.

   =============== ================================== ==================================
   ``sigma``       iteration mode                     ``which`` refers to eigenvalues of
   --------------- ---------------------------------- ----------------------------------
   ``nothing``     ordinary (forward)                 :math:`A`
   real or complex inverse with level shift ``sigma`` :math:`(A - \sigma I )^{-1}`
   =============== ================================== ==================================
```

eigs eigvecs eigvals eigvals! leading_ones leading_zeros

